![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

### Secure RESTful APIs using Python

# JSON Web Tokens for authentication

This project asks you to utilize JWTs for structured authentication rules.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1-5

**Future tokens**

Utilizing JWTs, and the PyJWT library specifically, we would like to create the following structured transaction:

1. A client requests a token from a server, and is verified using a public-key keyserver.
2. If client is a known user, it is granted a use token as a JWT.
3. The use token may not be used until 10 seconds after it is granted and expires 30 seconds after it is granted.
4. Any requests made to the root route are permitted in the authorized time window, but not outside it.  
5. Requests to `/` will fail, of course, if a use token is not provided.

For these tasks, you should run the identical `keyserver.py` service discussed in the lesson.  Once you have the server in the task running, you should be able to get responses similar to the below (responses are lines from The Zen of Python in the example; you may return whatever you like):

```python
from time import sleep
import requests
import jwt

client_private_key = open('client.key').read()
reqtoken = jwt.encode({'iss': "client"}, client_private_key, 
                      algorithm="RS256")
resp = requests.post('http://localhost:5025/login', data=reqtoken)
resptoken = resp.text
print(jwt.decode(resptoken, verify=False))

for _ in range(8):
    resp = requests.post('http://localhost:5025', data=resptoken)
    print(resp.status_code, resp.text)
    sleep(5)
```

```
{'nbf': 1619413878, 'exp': 1619413898}
403 The token is not yet valid (nbf)
403 The token is not yet valid (nbf)
200 Although never is often better than *right* now.
200 Now is better than never.
200 Although that way may not be obvious at first unless you're Dutch.
200 Now is better than never.
200 Explicit is better than implicit.
403 Signature has expired
```

This interaction should fail with appropriate HTTP status codes if the client does not provide valid credentials or if the wrong response token is provided.

**A possible solution**

Note that as in the lesson, we are omitting the TLS layer which a real implementation would require, to make the example easier.  A server that implements this might look like:

```python
#!/usr/bin/env python
from datetime import datetime, timedelta
from random import shuffle
from flask import Flask, abort, jsonify, request, make_response
import requests
import jwt
import zen
app = Flask(__name__)

keyserver = "http://localhost:5010/getkey"
server2_private_key = open('server2.key').read()
server2_public_key = open('server2.key.pub').read()

@app.route('/login', methods=['POST'])
def login():
    payload = jwt.decode(request.data, verify=False)
    user = payload.get('iss')
    resp = requests.get(f"{keyserver}?identity={user}")
    # Check if this is name of real user
    if resp.status_code != 200:
        abort(401)
    # Now find out if token really came from claimed user
    try:
        pubkey = resp.text
        jwt.decode(request.data, pubkey, algorithm="RS256")
        nbf = datetime.utcnow() + timedelta(seconds=10)
        exp = datetime.utcnow() + timedelta(seconds=30)
        
        token = jwt.encode({'nbf': nbf, 'exp': exp}, 
                           server2_private_key, algorithm="RS256")
        resp = make_response(token)
        resp.mimetype = "application/jwt"
        return resp
    except Exception as err:
        return make_response(str(err), 403)
    
@app.route('/', methods=['POST'])
def q():
    # Have I received a valid token I issues?
    try:
        jwt.decode(request.data, server2_public_key, algorithm="RS256")
        shuffle(zen.lines)
        return make_response(zen.lines[-1])
    except Exception as err:
        return make_response(str(err), 403)
         
if __name__ == "__main__":
    app.run(port=5025)
```

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)